In [1]:
# Step 1: Import the necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas_datareader import data as pdr

In [8]:
# Step 2: Load the data
file_path = 'C:\\Users\\phil7\\OneDrive\\Code\\_data\\ftw_spx_2024.csv'
strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)

C:\Users\phil7\AppData\Local\Temp\ipykernel_58816\3338920856.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)


In [9]:
# Step 3: Separate SPX and strategies
universe_prices = strategies_prices['SPX']
risk_premia_ftw_prices = strategies_prices.drop(columns=['SPX'])

# Convert the strategies to returns
strategies = risk_premia_ftw_prices.pct_change().dropna()
universe = universe_prices.pct_change().dropna()

In [ ]:
# The 4 strategies and the spx index are stored in a single csv file, the first column is the date, the second column is the first strategy, the third column is the second strategy, the fourth column is the third strategy, the fifth column is the fourth strategy, and the sixth column is the spx index
# The file is located in the following folder: C:\Users\phil7\OneDrive\Code\_data
# The name of the strategies are in the first row of the csv file
strategies_prices = pd.read_csv('C:\\Users\\phil7\\OneDrive\\Code\\_data\\ftw_spx_2024.csv', parse_dates=True)  # index_col='Date',
universe_prices = strategies_prices['SPX']
risk_premia_ftw_prices = strategies_prices.drop(columns=['SPX'])
# convert the strategies to returns
strategies = strategies_prices.pct_change().dropna()
universe = universe_prices.pct_change().dropna()
risk_premia_ftw = risk_premia_ftw_prices.pct_change().dropna()

In [10]:
# Step 4: Retrieve the Fama-French 4-factor data
ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]

# Combine the Fama-French factors and the momentum factor into a single DataFrame
ff_factors = ff_factors.join(momentum_factor['Mom   '])

# Convert the factors to daily returns as percentages
ff_factors = ff_factors / 100

C:\Users\phil7\AppData\Local\Temp\ipykernel_58816\3849217727.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
C:\Users\phil7\AppData\Local\Temp\ipykernel_58816\3849217727.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]


In [20]:
# Step 5: Run the Fama-French 4-factor regression for each strategy
model_results = {}
for strategy_name in strategies.columns:
    # Merge strategy returns with the Fama-French factors
    data = pd.concat([strategies[strategy_name], ff_factors], axis=1, join='inner')
    
    # Prepare the data for regression
    X = data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
    y = data[strategy_name] - data['RF']
    
    # Add a constant to the independent variables
    X = sm.add_constant(X)
    
    # Run the regression
    model = sm.OLS(y, X).fit()
    
    # Store the model results
    model_results[strategy_name] = model

In [21]:
# Step 6: Print the results
for strategy_name, result in model_results.items():
    print(f"\n{strategy_name} Regression Results:\n")
    print(result)


Quality Regression Results:


LowVol Regression Results:


Value Regression Results:


Momentum Regression Results:



In [25]:
coefficients = {}
p_values = {}
for strategy_name, model in model_results.items():
    coefficients[strategy_name] = model.params
    p_values[strategy_name] = model.pvalues

# Print the coefficients and p-values in the same table for each strategy
# Format for readability
coefficients_df = pd.DataFrame(coefficients).T
p_values_df = pd.DataFrame(p_values).T
coefficients_df.columns = ['Alpha', 'Mkt-RF', 'SMB', 'HML', 'Mom']
p_values_df.columns = ['Alpha', 'Mkt-RF', 'SMB', 'HML', 'Mom']
coefficients_df = coefficients_df.round(4)
p_values_df = p_values_df.applymap(lambda x: f"{x:.4f}")
print(coefficients_df)
print(p_values_df)

           Alpha  Mkt-RF     SMB     HML     Mom
Quality   0.0001  0.8282  0.0083  0.0256 -0.1420
LowVol    0.0010  0.4449 -0.1417 -0.0763 -0.5200
Value     0.0012  0.8370 -0.0037  0.5655 -0.4398
Momentum  0.0004  0.9663  0.1144  0.1026  0.2838
           Alpha  Mkt-RF     SMB     HML     Mom
Quality   0.8599  0.0000  0.8336  0.6453  0.0043
LowVol    0.0222  0.0000  0.0128  0.3276  0.0000
Value     0.0007  0.0000  0.9305  0.0000  0.0000
Momentum  0.2829  0.0000  0.0113  0.0998  0.0000


C:\Users\phil7\AppData\Local\Temp\ipykernel_58816\2412502736.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  p_values_df = p_values_df.applymap(lambda x: f"{x:.4f}")



SPX Regression Results:

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     5893.
Date:                Mon, 13 May 2024   Prob (F-statistic):           9.02e-73
Time:                        21:44:17   Log-Likelihood:                 401.79
No. Observations:                  61   AIC:                            -793.6
Df Residuals:                      56   BIC:                            -783.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.638e-05   4.